In [1]:
import random
import collections


# Configurations
LEVEL_MAPS = {
    'plains': ['fire', 'air', 'earth', 'water'],
    'volcano': ['fire', 'lightning', 'lava', 'steam'],
    'mountain': ['air', 'lightning', 'dust', 'ice'],
    'dessert': ['earth', 'lava', 'dust', 'grass'],
    'lake': ['water', 'steam', 'ice', 'grass'],
    'temple': ['light', 'dark']
}
POSSIBLE_MAPS = ['plains', 'volcano', 'mountain', 'dessert', 'lake']
MAP_MAIN = {'volcano': 'fire', 'mountain': 'air', 'dessert': 'earth', 'lake': 'water'}
LEVEL_CHOICES = {1:1, 2:1, 3:2, 4:3, 5:4, 6:3, 7:2, 8:1, 9:1}
LEVEL_APEX = {'volcano': 'water', 'mountain': 'fire', 'dessert': 'air', 'lake': 'earth'}
APEX_CHANCES = {3:0.1, 4:0.1, 5:0.2, 6:0.4}
LEVELS = 9

In [2]:
def generate_level_plains(level):
    this_map = 'plains'
    this_boss = random.choice(LEVEL_MAPS[this_map])
    return {'level': level, 'map': this_map, 'boss': this_boss, 'isApex': False, 'apexEd': False}

def generate_two_levels(prev_level):
    level = prev_level['level'] + 1
    possible_maps = list(set(POSSIBLE_MAPS) - {prev_level['map']})
    this_maps = random.sample(possible_maps, 2)
    map1 = this_maps[0]
    map2 = this_maps[1]
    boss1, is_apex1, apexed1 = generate_boss(level, map1, prev_level['apexEd'])
    boss2, is_apex2, apexed2 = generate_boss(level, map2, prev_level['apexEd'])
    level1 = {'level': level, 'map': map1, 'boss': boss1, 'isApex': is_apex1, 'apexEd': apexed1}
    level2 = {'level': level, 'map': map2, 'boss': boss2, 'isApex': is_apex2, 'apexEd': apexed2}
    return [level1, level2]

def generate_level_4(prev_levels):
    choice_b = generate_level_from_two(prev_levels[0], prev_levels[1])
    choice_a = generate_level(prev_levels[0], exclude_map=choice_b['map'])
    choice_c = generate_level(prev_levels[1], exclude_map=choice_b['map'])
    return [choice_a, choice_b, choice_c]

def generate_level_5(prev_levels):
    choice_b = generate_level_from_two(prev_levels[0], prev_levels[1])
    choice_c = generate_level_from_two(prev_levels[1], prev_levels[2])
    choice_a = generate_level(prev_levels[0], exclude_map=choice_b['map'])
    choice_d = generate_level(prev_levels[-1], exclude_map=choice_c['map'])
    return [choice_a, choice_b, choice_c, choice_d]

def generate_level_6(prev_levels):
    choice_a = generate_level_from_two(prev_levels[0], prev_levels[1])
    choice_b = generate_level_from_two(prev_levels[1], prev_levels[2], exclude_map=choice_a['map'])
    choice_c = generate_level_from_two(prev_levels[2], prev_levels[3], exclude_map=choice_b['map'])
    return [choice_a, choice_b, choice_c]

def generate_level_7(prev_levels):
    choice_a = generate_level_from_two(prev_levels[0], prev_levels[1])
    choice_b = generate_level_from_two(prev_levels[1], prev_levels[2], exclude_map=choice_a['map'])
    choice_a = generate_mythical_boss(choice_a)
    choice_b = generate_mythical_boss(choice_b)
    return [choice_a, choice_b]

def generate_level_8(prev_levels):
    this_level = generate_level_from_two(prev_levels[0], prev_levels[1])
    return generate_legendary_boss(this_level)

def generate_level(prev_level, exclude_map=None):
    level = prev_level['level'] + 1
    possible_maps = set(POSSIBLE_MAPS) - {prev_level['map']}
    if level >= 7:
        possible_maps = possible_maps - {'plains'}
    if exclude_map is not None:
        possible_maps = possible_maps - {exclude_map}
    this_map = random.choice(list(possible_maps))
    this_boss, is_apex, apexed = generate_boss(level, this_map, prev_level['apexEd'])
    return {'level': level, 'map': this_map, 'boss': this_boss, 'isApex': is_apex, 'apexEd': apexed}

def generate_level_from_two(prev1, prev2, exclude_map=None):
    level = prev1['level'] + 1
    possible_maps = set(POSSIBLE_MAPS) - {prev1['map'], prev2['map']}
    if level >= 7:
        possible_maps = possible_maps - {'plains'}
    if exclude_map is not None:
        possible_maps = possible_maps - {exclude_map}
    this_map = random.choice(list(possible_maps))
    prev_apexed = prev1['apexEd'] or prev2['apexEd']
    this_boss, is_apex, apexed = generate_boss(level, this_map, prev_apexed)
    return {'level': level, 'map': this_map, 'boss': this_boss, 'isApex': is_apex, 'apexEd': apexed}

def generate_boss(level, this_map, apexed):
    is_apex = check_apex(level, apexed)
    if is_apex:
        if this_map == 'plains':
            this_boss = random.choice(LEVEL_MAPS[this_map])
        else:
            this_boss = LEVEL_APEX[this_map]
        apexed = True
    else:
        this_boss = random.choice(LEVEL_MAPS[this_map])
    return this_boss, is_apex, apexed

def generate_mythical_boss(level):
    this_map = level['map']
    advanced = list(set(LEVEL_MAPS[this_map]) - {MAP_MAIN[this_map]})
    level['boss'] = random.choice(advanced)
    return level

def generate_legendary_boss(level):
    level['boss'] = MAP_MAIN[level['map']]
    return level

def check_apex(level, apexed):
    if level not in APEX_CHANCES.keys():
        return False
    apex_chance = APEX_CHANCES[level]
    if not apexed:
        apex_chance *= 2
    is_apex = random.random() < apex_chance
    return is_apex

In [3]:
def print_level(l, levels):
    print("Level", l)
    for level in levels:
        print("-" * l, level)
    print()

def generate_game():
    level1 = generate_level_plains(1)
    print_level(1, [level1])
    level2 = generate_level_plains(2)
    print_level(2, [level2])
    level3 = generate_two_levels(level2)
    print_level(3, level3)
    level4 = generate_level_4(level3)
    print_level(4, level4)
    level5 = generate_level_5(level4)
    print_level(5, level5)
    level6 = generate_level_6(level5)
    print_level(6, level6)
    level7 = generate_level_7(level6)
    print_level(7, level7)
    level8 = generate_level_8(level7)
    print_level(8, [level8])

In [4]:
generate_game()

Level 1
- {'level': 1, 'map': 'plains', 'boss': 'earth', 'isApex': False, 'apexEd': False}

Level 2
-- {'level': 2, 'map': 'plains', 'boss': 'fire', 'isApex': False, 'apexEd': False}

Level 3
--- {'level': 3, 'map': 'lake', 'boss': 'water', 'isApex': False, 'apexEd': False}
--- {'level': 3, 'map': 'dessert', 'boss': 'dust', 'isApex': False, 'apexEd': False}

Level 4
---- {'level': 4, 'map': 'volcano', 'boss': 'lightning', 'isApex': False, 'apexEd': False}
---- {'level': 4, 'map': 'mountain', 'boss': 'ice', 'isApex': False, 'apexEd': False}
---- {'level': 4, 'map': 'plains', 'boss': 'fire', 'isApex': False, 'apexEd': False}

Level 5
----- {'level': 5, 'map': 'plains', 'boss': 'fire', 'isApex': False, 'apexEd': False}
----- {'level': 5, 'map': 'lake', 'boss': 'earth', 'isApex': True, 'apexEd': True}
----- {'level': 5, 'map': 'lake', 'boss': 'earth', 'isApex': True, 'apexEd': True}
----- {'level': 5, 'map': 'mountain', 'boss': 'lightning', 'isApex': False, 'apexEd': False}

Level 6
------

In [5]:
generate_game()

Level 1
- {'level': 1, 'map': 'plains', 'boss': 'fire', 'isApex': False, 'apexEd': False}

Level 2
-- {'level': 2, 'map': 'plains', 'boss': 'fire', 'isApex': False, 'apexEd': False}

Level 3
--- {'level': 3, 'map': 'mountain', 'boss': 'lightning', 'isApex': False, 'apexEd': False}
--- {'level': 3, 'map': 'lake', 'boss': 'ice', 'isApex': False, 'apexEd': False}

Level 4
---- {'level': 4, 'map': 'lake', 'boss': 'earth', 'isApex': True, 'apexEd': True}
---- {'level': 4, 'map': 'plains', 'boss': 'air', 'isApex': False, 'apexEd': False}
---- {'level': 4, 'map': 'mountain', 'boss': 'lightning', 'isApex': False, 'apexEd': False}

Level 5
----- {'level': 5, 'map': 'volcano', 'boss': 'lightning', 'isApex': False, 'apexEd': True}
----- {'level': 5, 'map': 'mountain', 'boss': 'ice', 'isApex': False, 'apexEd': True}
----- {'level': 5, 'map': 'dessert', 'boss': 'air', 'isApex': True, 'apexEd': True}
----- {'level': 5, 'map': 'lake', 'boss': 'earth', 'isApex': True, 'apexEd': True}

Level 6
------ {

In [6]:
generate_game()

Level 1
- {'level': 1, 'map': 'plains', 'boss': 'earth', 'isApex': False, 'apexEd': False}

Level 2
-- {'level': 2, 'map': 'plains', 'boss': 'fire', 'isApex': False, 'apexEd': False}

Level 3
--- {'level': 3, 'map': 'dessert', 'boss': 'lava', 'isApex': False, 'apexEd': False}
--- {'level': 3, 'map': 'mountain', 'boss': 'air', 'isApex': False, 'apexEd': False}

Level 4
---- {'level': 4, 'map': 'mountain', 'boss': 'ice', 'isApex': False, 'apexEd': False}
---- {'level': 4, 'map': 'plains', 'boss': 'air', 'isApex': False, 'apexEd': False}
---- {'level': 4, 'map': 'lake', 'boss': 'grass', 'isApex': False, 'apexEd': False}

Level 5
----- {'level': 5, 'map': 'plains', 'boss': 'water', 'isApex': False, 'apexEd': False}
----- {'level': 5, 'map': 'dessert', 'boss': 'air', 'isApex': True, 'apexEd': True}
----- {'level': 5, 'map': 'volcano', 'boss': 'water', 'isApex': True, 'apexEd': True}
----- {'level': 5, 'map': 'dessert', 'boss': 'air', 'isApex': True, 'apexEd': True}

Level 6
------ {'level':